# Import Block

In [1]:
%pip install torch torchvision torchaudio python-dotenv lightning geopandas pandas sqlalchemy psycopg2-binary openpyxl geoalchemy2 python-dotenv numpy mlflow


Note: you may need to restart the kernel to use updated packages.


# Global Variables

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

import os
from dotenv import load_dotenv

import sys
src_path = "../../src/"
sys.path.append(src_path)
from utils.fed_env import FederatedEnvironment
from utils.dataset import Dataset
from utils.target_types import DTarget


In [3]:
PATH_TO_DOT_ENV = "../../.env"

DATABASE_TYPE = "postgresql"
DATABASE_HOST = "localhost"

MLFLOW_HOST = "localhost"


In [4]:
load_dotenv(PATH_TO_DOT_ENV)

DATABASE_NAME = os.environ.get("DATABASE_NAME")
POSTGRES_USER = os.environ.get("POSTGRES_USER")
POSTGRES_PASSWORD = os.environ.get("POSTGRES_PASSWORD")
POSTGRES_HOST_PORT = os.environ.get("POSTGRES_HOST_PORT")
POSTGRES_CONTAINER_PORT = os.environ.get("POSTGRES_CONTAINER_PORT")

MLFLOW_HOST_PORT = os.environ.get("MLFLOW_HOST_PORT")

In [5]:
N = 30
M = 1
K = 1

BATCH_SIZE = 4096
LEARNING_RATE = 0.01 
PHASE_3_LEARNING_RATE  = 0.001
LAYER_MULTIPYER = 1
RANDOM_STATE = 1
TEST_SIZE = 0.2
EPOCHS = 500
PHASE_1_EPOCHS = 50
NUM_NODES_PER_EPOCH = 1
NUM_TRAINING_PER_EPOCH = 1

MLFLOW_URI = f"http://{MLFLOW_HOST}:{MLFLOW_HOST_PORT}"

TARGET = DTarget.BOOLEAN
TYPE = Dataset.LIGHTNING

In [6]:
DATABASE_URL = URL.create(
    DATABASE_TYPE,
    username=POSTGRES_USER,
    password=POSTGRES_PASSWORD,  # plain (unescaped) text
    host=DATABASE_HOST,
    port=POSTGRES_HOST_PORT,
    database=DATABASE_NAME,
)
# engine = create_engine(DATABASE_URL)
# engine = create_engine(f"{DATABASE_TYPE}://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{DATABASE_HOST}:{POSTGRES_HOST_PORT}/{DATABASE_NAME}")

# Run Federated Learning

In [7]:

server = FederatedEnvironment(
    d_type=TYPE,
    d_target=TARGET,
    db_url=DATABASE_URL,
    mlflow_uri=MLFLOW_URI,
    n=N,
    m=M,
    k=K, 
    learning_rate=LEARNING_RATE,
    batch_size=BATCH_SIZE,
    multiplyer=LAYER_MULTIPYER
)
server.run_phase_1(
    epochs=PHASE_1_EPOCHS
)
server.fed_avg(
    epochs=EPOCHS, 
    num_nodes_per_epoch=NUM_NODES_PER_EPOCH,
    num_training_per_epoch=NUM_TRAINING_PER_EPOCH,
)

Cache of LIGHTNING_K1_N30_M1 found!
Cache of FIRE_K1_N30_M1 found!
Initializing id:   0
Initializing id:   5
Initializing id:   6
Initializing id:   7
Initializing id:  16
Initializing id:  18
Initializing id:  25
Initializing id:  27
Initializing id:  29
Initializing id:  51
Initializing id:  55
Initializing id:  58
Initializing id:  67
Initializing id:  68
Initializing id:  69
Initializing id:  70
Initializing id:  71
Initializing id:  72
Initializing id:  73
Started Phase 1...

Started updating nodes
Updating node 0
Updating node 5
Updating node 6
Updating node 7
Updating node 16
Updating node 18
Updating node 25
Updating node 27
Updating node 29
Updating node 51
Updating node 55
Updating node 58
Updating node 67
Updating node 68
Updating node 69
Updating node 70
Updating node 71
Updating node 72
Updating node 73



++++++++++++++++++++++++++   Training node  0   +++++++++++++++++++++++++++++++++++++++++++++++++++++


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2024/02/23 03:48:30 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
2024/02/23 03:48:33 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/mnt/2TB_HDD_00/Forest_Fire/federated-learning-framework/.venv/lib/python3.10/site-packages/mlflow/pytorch/_lightning_autolog.py:356: UserWarning: Autologging is known to be compatible with pytorch-lightning versions between 1.4.9 and 2.1.3 and may not succeed with packages outside this range."
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type          | Params
-------------------------------------------
0 | f1_score | BinaryF1Score | 0     
1 | model    | ModuleList    | 107 K 
----------

Epoch 49: 100%|██████████| 1/1 [00:00<00:00,  3.68it/s]                    

INFO: `Trainer.fit` stopped: `max_epochs=50` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 1/1 [00:00<00:00,  3.43it/s]


...

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Phase 2 testing node 27


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         f1_score          │    0.5492630004882812     │
│         val_loss          │    0.6515671014785767     │
└───────────────────────────┴───────────────────────────┘

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Phase 2 testing node 29
